# [IAPR 2020:][iapr2020] Project ‒ Special Project¶
**Author:** Quentin Talon & Albéric de Lajarte  
**Due date:** 28.05.2020  
[iapr2018]: https://github.com/LTS5/iapr-2020

## Extract datas

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pims

data_base_path = os.path.join(os.pardir, 'data')
vid = pims.open(os.path.join(data_base_path, 'robot_parcours_1.avi'))
plt.figure(figsize = (10,10))
plt.imshow(vid[0])
print(vid)

## Part 1: Still and moving image segmentation

### Background
We simply calculate the mean along all the images. This reduces the noise and removes the moving object.

In [ ]:
vid_stack = np.stack(vid, axis=0)
print("Shape of vid_stack:{}".format(vid_stack.shape))
background = np.median(vid_stack, axis=0).astype(int)
print("Shape of background:{}".format(background.shape))
plt.figure(figsize = (20,20))
plt.imshow(background)
plt.show()

### Moving part
This is simply a difference of image and the backgound. We look at the red channel and calculate it's center of mass.

In [ ]:
#Few tests.
from skimage.morphology import binary_opening, binary_closing, disk
from skimage.measure import regionprops
from scipy.ndimage.measurements import center_of_mass
red_threshold = 80#How to choose it ???
fig, ax = plt.subplots(1,2, figsize=(20,10))
red_dif = abs(vid[10] - background)[:,:,2]
foo = np.where(red_dif>red_threshold, 1, 0)
#foo = binary_opening(foo, selem=disk(2))
#foo = binary_closing(foo, selem=disk(2))
ax[0].imshow(red_dif)
ax[0].set_title("Red dif")
ax[1].imshow(foo)
ax[1].set_title("After binary operations, center of mass at:({:03.03f},{:03.03f})".format(*center_of_mass(foo)))
plt.show()

In [ ]:
trajectory = [center_of_mass(np.where(abs(v - background)[:,:,2]>red_threshold, 1, 0)) for v in vid]
plt.figure(figsize=(20,20))
plt.imshow(background)
for i, point in enumerate(trajectory):
    plt.plot(point[1], point[0], markeredgecolor='r', marker='x', markersize=20, markeredgewidth=5)
    plt.text(point[1]+10, point[0], "Frame#{}".format(i))
plt.text(100,400, "We can write our equation here", size=20, color='b')
plt.show()

## What to do now ?

In [ ]:
from skimage.measure import label
foo = np.all(np.where(background<150, 1, 0).astype(float), axis=2)#How to choose the treshold ?
foo = label(foo)
print(foo.shape)
plt.figure(figsize=(20,20))
plt.imshow(foo)